# Debug Drill: The Runaway Booster

**Scenario:**
A colleague trained a gradient boosting model for churn prediction. They followed "best practices" from a blog.

"I set n_estimators really high (1000) and the training loss keeps going down!" they say proudly.

But after deployment, the model performs terribly.

**Your Task:**
1. Run the model and diagnose the problem
2. Identify why the boosting overfit
3. Fix it with proper configuration
4. Write a 3-bullet postmortem

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

# Try to use XGBoost if available (faster, better early stopping)
try:
    from xgboost import XGBClassifier
    HAS_XGBOOST = True
    print("✓ Using XGBoost")
except ImportError:
    HAS_XGBOOST = False
    print("⚠ XGBoost not available, using sklearn GradientBoosting")

np.random.seed(42)

In [ ]:
# Load data
DATA_URL = 'https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/shared/data/'

customers = pd.read_csv(DATA_URL + 'streamcart_customers.csv')
print(f"Loaded {len(customers)} customers")

# Prepare features
if 'tenure_days' not in customers.columns:
    customers['tenure_days'] = (pd.to_datetime('2024-01-01') - pd.to_datetime(customers['signup_date'])).dt.days
if 'avg_order_value' not in customers.columns:
    customers['avg_order_value'] = customers['total_spend'] / customers['orders_total'].replace(0, 1)

feature_cols = ['tenure_days', 'orders_total', 'total_spend', 'support_tickets_total', 'avg_order_value']
available_features = [c for c in feature_cols if c in customers.columns]

X = customers[available_features].fillna(0)
y = customers['churn_30d']

# Train-validation-test split (need validation for early stopping)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

In [ ]:
# ===== COLLEAGUE'S CODE (CONTAINS BUGS) =====

# "I read that more trees is always better!"
# "And higher learning rate makes training faster!"
# "Deep trees capture complex patterns!"

if HAS_XGBOOST:
    gb_overfit = XGBClassifier(
        n_estimators=500,     # <-- BUG: Too many without early stopping
        max_depth=10,         # <-- BUG: Too deep for boosting
        learning_rate=0.5,    # <-- BUG: Too aggressive
        random_state=42,
        verbosity=0
    )
else:
    gb_overfit = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=10,
        learning_rate=0.5,
        random_state=42
    )

gb_overfit.fit(X_train, y_train)

train_acc = accuracy_score(y_train, gb_overfit.predict(X_train))
val_acc = accuracy_score(y_val, gb_overfit.predict(X_val))
test_acc = accuracy_score(y_test, gb_overfit.predict(X_test))

print("=== Colleague's Gradient Boosting ===")
print(f"  n_estimators: 500")
print(f"  max_depth: 10")
print(f"  learning_rate: 0.5")
print(f"\n  Train Accuracy: {train_acc:.1%}")
print(f"  Val Accuracy:   {val_acc:.1%}")
print(f"  Test Accuracy:  {test_acc:.1%}")
print(f"\n  Train-Test Gap: {train_acc - test_acc:.1%} ← OVERFITTING!")

---

## Your Investigation

The model memorized the training data. Let's understand why.

### Step 1: Understand what went wrong

In [ ]:
# The three deadly sins of boosting:
print("❌ The Three Deadly Sins of Boosting:")
print()
print("1. No early stopping:")
print("   - Boosting can perfectly fit training data given enough rounds")
print("   - Without stopping, it will overfit")
print()
print("2. High learning rate (0.5):")
print("   - Each tree makes large corrections")
print("   - Model fits noise quickly")
print(f"   - Colleague used: 0.5 (recommended: 0.01-0.1)")
print()
print("3. Deep trees (max_depth=10):")
print("   - Each tree is complex enough to overfit on its own")
print("   - Boosting works best with shallow 'weak' learners")
print(f"   - Colleague used: 10 (recommended: 3-6)")

In [ ]:
# Visualize how accuracy changes with number of rounds
# (This simulates what happens during training)

n_estimators_list = [10, 25, 50, 100, 200, 300, 400, 500]
results = []

for n in n_estimators_list:
    if HAS_XGBOOST:
        model = XGBClassifier(
            n_estimators=n, max_depth=10, learning_rate=0.5,
            random_state=42, verbosity=0
        )
    else:
        model = GradientBoostingClassifier(
            n_estimators=n, max_depth=10, learning_rate=0.5,
            random_state=42
        )
    model.fit(X_train, y_train)
    
    results.append({
        'n_estimators': n,
        'train_acc': accuracy_score(y_train, model.predict(X_train)),
        'val_acc': accuracy_score(y_val, model.predict(X_val)),
        'test_acc': accuracy_score(y_test, model.predict(X_test))
    })

results_df = pd.DataFrame(results)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(results_df['n_estimators'], results_df['train_acc'], 'b-o', label='Train', linewidth=2)
plt.plot(results_df['n_estimators'], results_df['val_acc'], 'g-s', label='Validation', linewidth=2)
plt.plot(results_df['n_estimators'], results_df['test_acc'], 'r-^', label='Test', linewidth=2)
plt.xlabel('Number of Boosting Rounds')
plt.ylabel('Accuracy')
plt.title('Overfitting: Train Accuracy Keeps Rising, Test Plateaus/Drops')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n🔍 Key observation: Validation accuracy peaks then drops!")
print("   This is classic overfitting. Early stopping would have caught this.")

In [ ]:
# Where would early stopping have stopped?
optimal_idx = results_df['val_acc'].idxmax()
optimal_n = results_df.loc[optimal_idx, 'n_estimators']

print(f"Early stopping would have stopped at: {optimal_n} rounds")
print(f"Colleague trained for: 500 rounds")
print(f"\nOvertraining by: {500 - optimal_n} rounds!")

### Step 2: Fix the configuration

In [ ]:
# TODO: Train a properly configured gradient boosting model

# Uncomment and complete:

# if HAS_XGBOOST:
#     gb_fixed = XGBClassifier(
#         n_estimators=500,        # Still high, but early stopping will save us
#         max_depth=4,             # FIX: Shallow trees (3-6 is typical)
#         learning_rate=0.1,       # FIX: Smaller steps (0.01-0.1 is typical)
#         early_stopping_rounds=20, # FIX: Stop if no improvement for 20 rounds
#         random_state=42,
#         verbosity=0
#     )
#     gb_fixed.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
#     stopped_at = gb_fixed.best_iteration
# else:
#     # sklearn GradientBoosting doesn't have built-in early stopping
#     # So we use the optimal n we found above
#     gb_fixed = GradientBoostingClassifier(
#         n_estimators=optimal_n,  # Use the optimal found earlier
#         max_depth=4,
#         learning_rate=0.1,
#         random_state=42
#     )
#     gb_fixed.fit(X_train, y_train)
#     stopped_at = optimal_n
# 
# train_acc_fixed = accuracy_score(y_train, gb_fixed.predict(X_train))
# val_acc_fixed = accuracy_score(y_val, gb_fixed.predict(X_val))
# test_acc_fixed = accuracy_score(y_test, gb_fixed.predict(X_test))
# 
# print("=== Fixed Gradient Boosting ===")
# print(f"  Stopped at round: {stopped_at}")
# print(f"  max_depth: 4")
# print(f"  learning_rate: 0.1")
# print(f"\n  Train Accuracy: {train_acc_fixed:.1%}")
# print(f"  Val Accuracy:   {val_acc_fixed:.1%}")
# print(f"  Test Accuracy:  {test_acc_fixed:.1%}")
# print(f"  Train-Test Gap: {train_acc_fixed - test_acc_fixed:.1%}")

In [ ]:
# TODO: Compare the models

# Uncomment:

# print("\n=== Comparison ===")
# print(f"                    Overfit Model   Fixed Model")
# print(f"  Rounds:           {500:>12}    {stopped_at:>12}")
# print(f"  max_depth:        {10:>12}    {4:>12}")
# print(f"  learning_rate:    {0.5:>12}    {0.1:>12}")
# print(f"  Train Acc:        {train_acc:>12.1%}    {train_acc_fixed:>12.1%}")
# print(f"  Test Acc:         {test_acc:>12.1%}    {test_acc_fixed:>12.1%}")
# print(f"  Gap:              {train_acc - test_acc:>12.1%}    {train_acc_fixed - test_acc_fixed:>12.1%}")
# print(f"\n  Test improvement: +{test_acc_fixed - test_acc:.1%}")

In [ ]:
# ============================================
# SELF-CHECK: Did you fix the overfitting?
# ============================================

# Uncomment:

# assert (train_acc_fixed - test_acc_fixed) < 0.15, "Train-test gap should be reduced"
# assert test_acc_fixed >= test_acc - 0.02, "Test accuracy should not drop significantly"
# assert stopped_at < 500, "Early stopping should have stopped before 500 rounds"
# 
# print("✓ Boosting overfitting fixed!")
# print(f"✓ Stopped at {stopped_at} rounds instead of 500")
# print(f"✓ Train-test gap reduced from {train_acc - test_acc:.1%} to {train_acc_fixed - test_acc_fixed:.1%}")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Runaway Booster

### What happened:
- (Your answer: What symptoms indicated overfitting?)

### Root cause:
- (Your answer: Which three configuration issues caused overfitting?)

### How to prevent:
- (Your answer: What should ALWAYS be used with gradient boosting?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Always use early stopping with boosting.** It's the single most important regularization technique.

2. **Keep trees shallow (depth 3-6).** Boosting works by combining many weak learners, not one strong one.

3. **Use small learning rates (0.01-0.1).** Slower learning with more rounds generalizes better.

4. **Monitor validation loss.** When it stops improving, stop training.

---

## Boosting Configuration Cheatsheet

| Parameter | Dangerous | Safe | Notes |
|-----------|-----------|------|-------|
| n_estimators | 1000+ without early stopping | 100-500 with early stopping | More is fine IF using early stopping |
| learning_rate | 0.3+ | 0.01-0.1 | Lower is safer, needs more rounds |
| max_depth | 8+ | 3-6 | Deep trees overfit quickly |
| early_stopping | Off | On (10-50 rounds) | ALWAYS use with boosting |